<center>
    <p align="center">
        <img src="https://logodownload.org/wp-content/uploads/2017/09/mackenzie-logo-3.png" style="height: 7ch;"><br>
        <h1 align="center">Computer Systems Undergradute Thesis</h1>
        <h2 align="center">Quantitative Analysis of the Impact of Image Pre-Processing on the Accuracy of Computer Vision Models Trained to Identify Dermatological Skin Diseases</a>
        <h4 align="center">Gabriel Mitelman Tkacz</a>
        </h4>
    </p>
</center>

<hr>

In [1]:
import re
import tomllib
from functools import partial
from itertools import permutations
from pprint import pprint

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from pynimbar import loading_animation

from util import (ColorSpaceTransform, DenoiseTransform, EqualizationTransform,
                  NormalizeTransform, evaluate_model, get_model_data)

In [20]:
with open("parameters.toml", "r") as f:
    parameters = tomllib.loads(f.read())

loading_handler = partial(
    loading_animation, break_on_error=True, verbose_errors=True, time_it_live=True
)

alpha = chr(0x03B1)

pd.set_option('display.max_rows', 500)

pprint(parameters)

{'PREPROCESS': {'colorspace': {'source_space': 'RGB', 'target_space': 'HSV'},
                'denoise': {'search_window_size': 19,
                            'template_window_size': 5},
                'normalize': {'mean': 0.4, 'std': 0.2}},
 'TRAINING': {'batch_size': 128,
              'diseased_skin_path': './dataset/diseased/',
              'healthy_skin_path': './dataset/healthy/',
              'learning_rate': 0.0001,
              'num_epochs': 3,
              'num_workers': 12,
              'pin_memory': True,
              'precision_threshold': 0.8,
              'resize_dim': 128,
              'shuffle': True,
              'training_dataset_ratio': 0.8}}


In [3]:
preprocesses = (
    ColorSpaceTransform(**parameters["PREPROCESS"]["colorspace"]),
    DenoiseTransform(**parameters["PREPROCESS"]["denoise"]),
    EqualizationTransform(),
    NormalizeTransform(**parameters["PREPROCESS"]["normalize"]),
)

preprocess_combinations = {
    i: permutations(preprocesses, i) for i in range(2, len(preprocesses) + 1)
}

preprocess_labels = {s.__class__.__name__: re.sub('[^A-Z]', '', s.__class__.__name__)[:-1] for s in preprocesses}
preprocess_labels

{'ColorSpaceTransform': 'CS',
 'DenoiseTransform': 'D',
 'EqualizationTransform': 'E',
 'NormalizeTransform': 'N'}

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
training_ratio = parameters["TRAINING"]["training_dataset_ratio"]
testing_ratio = validation_ratio = round(1 - training_ratio, 1) / 2

print(f"Training ratio: {training_ratio*100}%")
print(f"Testing ratio: {testing_ratio*100}%")
print(f"Validation ratio: {validation_ratio*100}%")

seed = 47
print(f"\nSeed: {seed}")

Training ratio: 80.0%
Testing ratio: 10.0%
Validation ratio: 10.0%

Seed: 47


## Class 0 Model: Images with no pre-processing

In [6]:
(
    base_train_loader,
    base_test_loader,
    base_validation_loader,
) = get_model_data(
    training_ratio=training_ratio,
    testing_ratio=testing_ratio,
    validation_ratio=validation_ratio,
    seed=seed,
)

base_precision, base_confusion_matrix = evaluate_model(
    device, base_train_loader, base_test_loader, base_validation_loader
)

print(f"Base precision: {base_precision*100:.1f}%")

if base_precision < parameters["TRAINING"]["precision_threshold"]:
    raise ValueError("The base model did not meet the precision threshold.")

base_confusion_matrix

Epoch 1/3, Train Loss: 0.0488, Train Accuracy: 93.9%, Validation Loss: 0.0427, Validation Accuracy: 94.4%
Epoch 2/3, Train Loss: 0.0223, Train Accuracy: 97.2%, Validation Loss: 0.0221, Validation Accuracy: 97.2%
Epoch 3/3, Train Loss: 0.0204, Train Accuracy: 97.5%, Validation Loss: 0.0231, Validation Accuracy: 96.8%
Total training duration: 3.6 minutes
Test Accuracy of the Binary Classification Model: 97.3%
Base precision: 97.3%


{'TP': 369, 'TN': 604, 'FP': 8, 'FN': 19}

## Class 1 Models: Images with only one pre-process

### Class 1.1 Models: Normalizing the image

In [ ]:
(
    normalize_train_loader,
    normalize_test_loader,
    normalize_validation_loader,
) = get_model_data(
    [NormalizeTransform(**parameters["PREPROCESS"]["normalize"])],
    training_ratio=training_ratio,
    testing_ratio=testing_ratio,
    validation_ratio=validation_ratio,
    seed=seed,
)

normalize_precision = evaluate_model(
    device, normalize_train_loader, normalize_test_loader, normalize_validation_loader
)

normalize_precision_diff = normalize_precision - base_precision

print(f"\n\nNormalized precision: {normalize_precision*100:.1f}%")
print(
    f"That is an {'upgrade' if normalize_precision_diff > 0 else 'downgrade'} of {normalize_precision_diff*100:.1f}%."
)

In [ ]:
df = pd.read_json("./params/normalize.json")

df["precision"] = df["precision"] - base_precision

pivot_table = df.pivot(index="std", columns="mean", values="precision")

pivot_table = pivot_table.sort_index().sort_index(axis=1).iloc[::-1]

pivot_table_pct = pivot_table * 100

vabs = max(abs(pivot_table_pct.min().min()), abs(pivot_table_pct.max().max()))

plt.figure(figsize=(8, 6))

sns.heatmap(
    pivot_table_pct,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    cbar_kws={"label": alpha, "format": "%.0f%%"},
    vmin=-vabs,
    vmax=vabs,
)

plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.gca().set_xticklabels([f"{x:.1f}" for x in pivot_table_pct.columns])
plt.gca().set_yticklabels([f"{y:.1f}" for y in pivot_table_pct.index])

plt.title("Correlation between Normalization Parameters and Model Precision")
plt.xlabel("Mean")
plt.ylabel("Standard Deviation")

plt.show()

### Class 1.2 Models: Denoising the image

In [ ]:
(
    denoise_train_loader,
    denoise_test_loader,
    denoise_validation_loader,
) = get_model_data(
    [DenoiseTransform(**parameters["PREPROCESS"]["denoise"])],
    training_ratio=training_ratio,
    testing_ratio=testing_ratio,
    validation_ratio=validation_ratio,
    seed=seed,
)

denoise_precision = evaluate_model(
    device, denoise_train_loader, denoise_test_loader, denoise_validation_loader
)

denoise_precision_diff = denoise_precision - base_precision

print(f"\n\nDenoised precision: {denoise_precision*100:.1f}%")
print(
    f"That is an {'upgrade' if denoise_precision_diff > 0 else 'downgrade'} of {denoise_precision_diff*100:.1f}%."
)

In [ ]:
df = pd.read_json("./params/denoise.json")

df["precision"] = df["precision"] - base_precision

pivot_table = df.pivot(
    index="search_window_size", columns="template_window_size", values="precision"
)

pivot_table = pivot_table.sort_index().sort_index(axis=1).iloc[::-1]

pivot_table_pct = pivot_table * 100

vabs = max(abs(pivot_table_pct.min().min()), abs(pivot_table_pct.max().max()))

plt.figure(figsize=(8, 6))

sns.heatmap(
    pivot_table_pct,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    cbar_kws={"label": alpha, "format": "%.0f%%"},
    vmin=-vabs,
    vmax=vabs,
)

plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.gca().set_xticklabels([f"{x:.0f}" for x in pivot_table_pct.columns])
plt.gca().set_yticklabels([f"{y:.0f}" for y in pivot_table_pct.index])

plt.title("Correlation between Denoising Parameters and Model Precision")
plt.xlabel("Template Window Size")
plt.ylabel("Search Window Size")

plt.show()

### Class 1.3 Models: Equalizing the image

In [ ]:
(
    equalized_train_loader,
    equalized_test_loader,
    equalized_validation_loader,
) = get_model_data(
    [EqualizationTransform()],
    training_ratio=training_ratio,
    testing_ratio=testing_ratio,
    validation_ratio=validation_ratio,
    seed=seed,
)

equalized_precision = evaluate_model(
    device, equalized_train_loader, equalized_test_loader, equalized_validation_loader
)

equalized_precision_diff = equalized_precision - base_precision

print(f"\n\nEqualized precision: {equalized_precision*100:.1f}%")
print(
    f"That is an {'upgrade' if equalized_precision_diff > 0 else 'downgrade'} of {equalized_precision_diff*100:.1f}%."
)

### Class 1.4 Models: Changing the colorspace

In [ ]:
(
    colorspace_train_loader,
    colorspace_test_loader,
    colorspace_validation_loader,
) = get_model_data(
    [ColorSpaceTransform(**parameters["PREPROCESS"]["colorspace"])],
    training_ratio=training_ratio,
    testing_ratio=testing_ratio,
    validation_ratio=validation_ratio,
    seed=seed,
)

colorspace_precision = evaluate_model(
    device, colorspace_train_loader, colorspace_test_loader, colorspace_validation_loader
)

colorspace_precision_diff = colorspace_precision - base_precision

print(f"\n\nColorspaced precision: {colorspace_precision*100:.1f}%")
print(
    f"That is an {'upgrade' if colorspace_precision_diff > 0 else 'downgrade'} of {colorspace_precision_diff*100:.1f}%."
)

In [ ]:
df = pd.read_json("./params/colorspace.json")

df["precision"] = df["precision"] - base_precision

pivot_table = df.set_index("target_space")

pivot_table = pivot_table.sort_index().sort_index(axis=1).iloc[::-1]

pivot_table_pct = pivot_table * 100

vabs = max(abs(pivot_table_pct.min().min()), abs(pivot_table_pct.max().max()))

plt.figure(figsize=(8, 6))

sns.heatmap(
    pivot_table_pct,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    cbar_kws={"label": alpha, "format": "%.0f%%"},
    vmin=-vabs,
    vmax=vabs,
)

plt.xticks(rotation=0)
plt.yticks(rotation=0)

plt.title("Correlation between Color Space Parameters and Model Precision")
plt.ylabel("Target Color Space")

plt.show()

In [14]:
class1_precisions = {
    NormalizeTransform.__name__: normalize_precision,
    EqualizationTransform.__name__: equalized_precision,
    DenoiseTransform.__name__: denoise_precision,
    ColorSpaceTransform.__name__: colorspace_precision,
}

class1_df_data = [
    {
        "transform_1": k.split(", ")[0],
        "precision": v,
        alpha: v - base_precision,
    }
    for k, v in class1_precisions.items()
]
class1_df = pd.DataFrame(class1_df_data).sort_values(alpha, ascending=False).reset_index(drop=True)
class1_df.transform_1 = class1_df.transform_1.apply(lambda x: preprocess_labels[x])
class1_df

,transform_1,precision,α
0,N,0.955,0.060
1,E,0.945,0.050
2,D,0.940,0.045
3,CS,0.915,0.020


## Class 2 Models: Images with two pre-processes

In [ ]:
class2_precisions = {}

for idx, combination in enumerate(preprocess_combinations[2]):
    (
        class2_train_loader,
        class2_test_loader,
        class2_validation_loader,
    ) = get_model_data(
        combination,
        training_ratio=training_ratio,
        testing_ratio=testing_ratio,
        validation_ratio=validation_ratio,
        seed=seed,
    )

    curr_precision = evaluate_model(
        device, class2_train_loader, class2_test_loader, class2_validation_loader, verbose=False
    )
    
    uuid = ", ".join([str(t.__class__.__name__) for t in combination])

    class2_precisions[uuid] = curr_precision

    curr_precision_diff = curr_precision - base_precision

    print(f"\n\nClass 2.{idx+1} {uuid} precision: {curr_precision*100:.1f}%")
    print(
        f"That is an {'upgrade' if curr_precision_diff > 0 else 'downgrade'} of {curr_precision_diff*100:.1f}%."
    )

In [ ]:
class2_df_data = [
    {
        "transform_1": k.split(", ")[0],
        "transform_2": k.split(", ")[1],
        "precision": v,
        alpha: v - base_precision,
    }
    for k, v in class2_precisions.items()
]
class2_df = pd.DataFrame(class2_df_data).sort_values(alpha, ascending=False).reset_index(drop=True)
class2_df.transform_1 = class2_df.transform_1.apply(lambda x: preprocess_labels[x])
class2_df.transform_2 = class2_df.transform_2.apply(lambda x: preprocess_labels[x])
class2_df

In [ ]:
grouped = class2_df.groupby(['transform_1', 'transform_2']).mean()
grouped[alpha] *= 100

vabs = max(abs(grouped.min().min()), abs(grouped.max().max()))

pivot = grouped[alpha].unstack()

plt.figure(figsize=(8, 6))
sns.heatmap(
    pivot,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    cbar_kws={"label": alpha, "format": "%.0f%%"},
    vmax=vabs,
    vmin=-vabs
)
plt.title('Model Precision by Transform Combinations')
plt.ylabel('Transform 1')
plt.xlabel('Transform 2')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

## Class 3 Models: Images with three pre-processes

In [ ]:
class3_precisions = {}

for idx, combination in enumerate(preprocess_combinations[3]):
    (
        class3_train_loader,
        class3_test_loader,
        class3_validation_loader,
    ) = get_model_data(
        combination,
        training_ratio=training_ratio,
        testing_ratio=testing_ratio,
        validation_ratio=validation_ratio,
        seed=seed,
    )

    curr_precision = evaluate_model(
        device, class3_train_loader, class3_test_loader, class3_validation_loader, verbose=False
    )
    
    uuid = "→".join([str(t.__class__.__name__) for t in combination])

    class3_precisions[uuid] = curr_precision

    curr_precision_diff = curr_precision - base_precision

    print(f"\n\nClass 3.{idx+1} {uuid} precision: {curr_precision*100:.1f}%")
    print(
        f"That is an {'upgrade' if curr_precision_diff > 0 else 'downgrade'} of {curr_precision_diff*100:.1f}%."
    )

In [10]:
class3_df_data = [
    {
        "transform_1": k.split("→")[0],
        "transform_2": k.split("→")[1],
        "transform_3": k.split("→")[2],
        "precision": v,
        alpha: v - base_precision,
    }
    for k, v in class3_precisions.items()
]
class3_df = pd.DataFrame(class3_df_data).sort_values(alpha, ascending=False).reset_index(drop=True)
class3_df.transform_1 = class3_df.transform_1.apply(lambda x: preprocess_labels[x])
class3_df.transform_2 = class3_df.transform_2.apply(lambda x: preprocess_labels[x])
class3_df.transform_3 = class3_df.transform_3.apply(lambda x: preprocess_labels[x])
class3_df

,transform_1,transform_2,transform_3,precision,α
0,D,E,N,0.960,0.065
1,E,D,N,0.955,0.060
2,E,N,D,0.955,0.060
3,CS,D,N,0.925,0.030
4,E,CS,N,0.920,0.025
5,D,CS,N,0.915,0.020
6,E,D,CS,0.915,0.020
7,E,CS,D,0.905,0.010
8,CS,E,N,0.885,-0.010
9,D,CS,E,0.885,-0.010


In [ ]:
order = preprocess_labels.values()

# Set up the matplotlib figure
plt.figure(figsize=(18, 6))

# Heatmap for Transform 1 vs Alpha
plt.subplot(1, 3, 1)
# Calculate mean alpha for each category in transform_1
pivot_t1 = class3_df.pivot_table(values=alpha, index='transform_1', aggfunc='mean').reindex(order)
# Transpose for heatmap
pivot_t1 = pivot_t1.T
sns.heatmap(pivot_t1, annot=True, cmap='RdYlGn', cbar=False)
plt.title('Impact of Transform 1 on Alpha')
plt.xlabel('Transform 1')
plt.yticks([])  # Hide y-axis labels

# Heatmap for Transform 2 vs Alpha
plt.subplot(1, 3, 2)
# Calculate mean alpha for each category in transform_2
pivot_t2 = class3_df.pivot_table(values=alpha, index='transform_2', aggfunc='mean').reindex(order)
# Transpose for heatmap
pivot_t2 = pivot_t2.T
sns.heatmap(pivot_t2, annot=True, cmap='RdYlGn', cbar=False)
plt.title('Impact of Transform 2 on Alpha')
plt.xlabel('Transform 2')
plt.yticks([])  # Hide y-axis labels

# Heatmap for Transform 3 vs Alpha
plt.subplot(1, 3, 3)
# Calculate mean alpha for each category in transform_3
pivot_t3 = class3_df.pivot_table(values=alpha, index='transform_3', aggfunc='mean').reindex(order)
# Transpose for heatmap
pivot_t3 = pivot_t3.T
sns.heatmap(pivot_t3, annot=True, cmap='RdYlGn', cbar=False)
plt.title('Impact of Transform 3 on Alpha')
plt.xlabel('Transform 3')
plt.yticks([])  # Hide y-axis labels

plt.tight_layout()
plt.show()

# Additional Heatmaps for Combined Transforms

# Heatmap for Transform 1 and Transform 2
# plt.figure(figsize=(12, 6))
# pivot_t1_t2 = class3_df.pivot_table(values=alpha, index='transform_1', columns='transform_2', aggfunc='mean').reindex(index=order, columns=order)
# sns.heatmap(pivot_t1_t2, annot=True, cmap='RdYlGn', linewidths=0.5, linecolor='gray')
# plt.title('Impact of Transform 1 and Transform 2 on Alpha')
# plt.xlabel('Transform 2')
# plt.ylabel('Transform 1')
# plt.show()

# # Heatmap for Transform 1 and Transform 3
# plt.figure(figsize=(12, 6))
# pivot_t1_t3 = class3_df.pivot_table(values=alpha, index='transform_1', columns='transform_3', aggfunc='mean').reindex(index=order, columns=order)
# sns.heatmap(pivot_t1_t3, annot=True, cmap='RdYlGn', linewidths=0.5, linecolor='gray')
# plt.title('Impact of Transform 1 and Transform 3 on Alpha')
# plt.xlabel('Transform 3')
# plt.ylabel('Transform 1')
# plt.show()

# # Heatmap for Transform 2 and Transform 3
# plt.figure(figsize=(12, 6))
# pivot_t2_t3 = class3_df.pivot_table(values=alpha, index='transform_2', columns='transform_3', aggfunc='mean').reindex(index=order, columns=order)
# sns.heatmap(pivot_t2_t3, annot=True, cmap='RdYlGn', linewidths=0.5, linecolor='gray')
# plt.title('Impact of Transform 2 and Transform 3 on Alpha')
# plt.xlabel('Transform 3')
# plt.ylabel('Transform 2')
# plt.show()


## Class 4 Models: Images with four pre-processes

In [ ]:
class4_precisions = {}

for idx, combination in enumerate(preprocess_combinations[4]):
    (
        class4_train_loader,
        class4_test_loader,
        class4_validation_loader,
    ) = get_model_data(
        combination,
        training_ratio=training_ratio,
        testing_ratio=testing_ratio,
        validation_ratio=validation_ratio,
        seed=seed,
    )

    curr_precision = evaluate_model(
        device, class4_train_loader, class4_test_loader, class4_validation_loader, verbose=False
    )
    
    uuid = "➔".join([str(t.__class__.__name__) for t in combination])

    class4_precisions[uuid] = curr_precision

    curr_precision_diff = curr_precision - base_precision

    print(f"\n\nClass 4.{idx+1} {uuid} precision: {curr_precision*100:.1f}%")
    print(
        f"That is an {'upgrade' if curr_precision_diff > 0 else 'downgrade'} of {curr_precision_diff*100:.1f}%."
    )

In [9]:
class4_df_data = [
    {
        "transform_1": k.split("➔")[0],
        "transform_2": k.split("➔")[1],
        "transform_3": k.split("➔")[2],
        "transform_4": k.split("➔")[3],
        "precision": v,
        alpha: v - base_precision,
    }
    for k, v in class4_precisions.items()
]
class4_df = pd.DataFrame(class4_df_data).sort_values(alpha, ascending=False).reset_index(drop=True)
class4_df.transform_1 = class4_df.transform_1.apply(lambda x: preprocess_labels[x])
class4_df.transform_2 = class4_df.transform_2.apply(lambda x: preprocess_labels[x])
class4_df.transform_3 = class4_df.transform_3.apply(lambda x: preprocess_labels[x])
class4_df.transform_4 = class4_df.transform_4.apply(lambda x: preprocess_labels[x])
class4_df

,transform_1,transform_2,transform_3,transform_4,precision,α
0,CS,E,D,N,0.940,0.045
1,E,D,CS,N,0.940,0.045
2,D,E,CS,N,0.935,0.040
3,E,CS,D,N,0.930,0.035
4,D,CS,E,N,0.915,0.020
5,CS,D,E,N,0.895,0.000
6,E,N,CS,D,0.885,-0.010
7,E,N,D,CS,0.885,-0.010
8,CS,E,N,D,0.875,-0.020
9,E,D,N,CS,0.850,-0.045


In [21]:
analysis_df = pd.concat([class1_df, class2_df, class3_df, class4_df], axis=0).reset_index(drop=True)[['transform_1', 'transform_2', 'transform_3', 'transform_4', 'precision', alpha]]
analysis_df.head(100)

,transform_1,transform_2,transform_3,transform_4,precision,α
0,N,NaN,NaN,NaN,0.955,0.060
1,E,NaN,NaN,NaN,0.945,0.050
2,D,NaN,NaN,NaN,0.940,0.045
3,CS,NaN,NaN,NaN,0.915,0.020
4,D,E,NaN,NaN,0.960,0.065
5,D,N,NaN,NaN,0.950,0.055
6,E,N,NaN,NaN,0.950,0.055
7,D,CS,NaN,NaN,0.930,0.035
8,CS,N,NaN,NaN,0.925,0.030
9,CS,D,NaN,NaN,0.910,0.015


In [7]:
import dill
filename = 'globalsave.pkl'
# dill.dump_session(filename)
dill.load_session(filename)